# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [35]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [36]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [37]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [38]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [39]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [40]:
# Create a Keyspace udacity
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [41]:
# Set KEYSPACE to the keyspace udacity
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




# Query 1 - Create the table
Create the table that will store the information needed for the query 1

In [42]:
# Drop table if exists song_info_by_session (needed in case of multiple attempts for achieving the best model)
query = "DROP TABLE IF EXISTS song_info_by_session"
try:
    session.execute(query)
except Exception as e:
    print(e)

# Create table song_info_by_session with the columns available in event_datafile_new.csv file,
# defining as primary key: session_id(partition key) and item_in_session(clustering column)
query = """
    CREATE TABLE IF NOT EXISTS song_info_by_session 
        (session_id int,  
        item_in_session int, 
        artist text, 
        song text, 
        length float, 
        first_name text, 
        gender text,
        last_name text, 
        level text, 
        location text, 
        user_id int, 
        PRIMARY KEY(session_id, item_in_session))
        WITH CLUSTERING ORDER BY (item_in_session DESC);
"""
try:
    session.execute(query)
except Exception as e:
    print(e)

# Query 1 - Insert the data into the table
Move the data that is located in the csv file into the sql table

In [43]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = """
            INSERT INTO song_info_by_session 
                (session_id,  
                item_in_session, 
                artist, 
                song, 
                length, 
                first_name, 
                gender,
                last_name, 
                level, 
                location, 
                user_id) 
            VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5]), line[1], line[2], line[4], line[6], line[7], int(line[10])))

# Query 1 - Solution: Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [44]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query = """
    SELECT 
        artist, 
        song,
        length 
    FROM song_info_by_session 
    WHERE session_id = 338 AND 
        item_in_session = 4
    """
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.artist, "|", row.song, "|", row.length)
                    

Faithless | Music Matters (Mark Knight Dub) | 495.30731201171875


# Query 2 - Create the table
Create the table that will store the information needed for the query 2

In [45]:
# Drop table if exists user_preferences_by_user_and_session (needed in case of multiple attempts for achieving the best model)
query = "DROP TABLE IF EXISTS user_preferences_by_user_and_session"
try:
    session.execute(query)
except Exception as e:
    print(e)

# Create table user_preferences_by_user_and_session with the columns available in event_datafile_new.csv file,
# defining as primary key: user_id(partition key), item_in_session(clustering column), and session_id(clustering column)
# As the sorting is done by the primary key, although item_in_session is the last clustering column, 
# it will be sorted correctly becasue session_id and user_id do not interfere in the sorting preferences
query = """
    CREATE TABLE IF NOT EXISTS user_preferences_by_user_and_session
        (session_id int,
        user_id int,
        item_in_session int,
        artist text,
        song text,
        first_name text,
        last_name text,
        gender text,
        length float,
        level text, 
        location text,
        PRIMARY KEY((user_id, session_id), item_in_session))
        
"""

try:
    session.execute(query)
except Exception as e:
    print(e)

# Query 2 - Insert the data into the table
Move the data that is located in the csv file into the sql table

In [46]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = """
            INSERT INTO user_preferences_by_user_and_session 
                (session_id,
                user_id,
                item_in_session,
                artist,
                song,
                first_name,
                last_name,
                gender,
                length,
                level, 
                location) 
            VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[10]), int(line[3]), line[0], line[9], line[1], line[4], line[2], float(line[5]), line[6], line[7]))

# Query 2 - Solution: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [47]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query = """
    SELECT artist, 
        song, 
        first_name, 
        last_name 
    FROM user_preferences_by_user_and_session 
    WHERE session_id = 182 AND 
        user_id = 10
    """
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.artist, "|", row.song, "|", row.first_name, "|", row.last_name)

                    

Down To The Bone | Keep On Keepin' On | Sylvie | Cruz
Three Drives | Greece 2000 | Sylvie | Cruz
Sebastien Tellier | Kilometer | Sylvie | Cruz
Lonnie Gordon | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) | Sylvie | Cruz


# Query 3 - Create the table
Create the table that will store the information needed for the query 3

In [48]:
## Create table music_history_songs
query = "DROP TABLE IF EXISTS user_by_song"
try:
    session.execute(query)
except Exception as e:
    print(e)

# Create table user_preferences_by_user_and_session with the columns available in event_datafile_new.csv file,
# defining as primary key: song(partition key) and user_id(clustering column)
query = """
    CREATE TABLE IF NOT EXISTS user_by_song
        (song text, 
        user_id int, 
        first_name text, 
        last_name text, 
        artist text, 
        gender text, 
        item_in_session int, 
        length float, 
        level text, 
        location text, 
        session_id int, 
        PRIMARY KEY(song, user_id))
"""
try:
    session.execute(query)
except Exception as e:
    print(e)

# Query 3 - Insert the data into the table
Move the data that is located in the csv file into the sql table

In [49]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = """
            INSERT INTO user_by_song
                (song, 
                user_id, 
                first_name, 
                last_name, 
                artist, 
                gender, 
                item_in_session, 
                length, 
                level, 
                location, 
                session_id) 
            VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[9], int(line[10]), line[1], line[4], line[0], line[2], int(line[3]), float(line[5]), line[6], line[7], int(line[8])))

# Query 3 - Solution: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [50]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = """
    SELECT first_name, 
        last_name 
    FROM user_by_song 
    WHERE song = 'All Hands Against His Own'
    """
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.first_name, "|", row.last_name)

                    

Jacqueline | Lynch
Tegan | Levine
Sara | Johnson


### Drop the tables before closing out the sessions

In [51]:
## Drop the tables before closing out the sessions
query = "DROP TABLE IF EXISTS song_info_by_session"
try:
    session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE IF EXISTS user_preferences_by_user_and_session"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE IF EXISTS user_by_song"
try:
    session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [52]:
session.shutdown()
cluster.shutdown()